# 4.9 Part 1

## Script Points

1. Import Data & Libraries
2. Data Wrangling
3. Data Consistency Checks
4. Combine Data
5. Exporting

# Import Libraries & Data

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Creat Path

path = r'C:\Users\alexi\OneDrive\02-2023 Instacart Basket Analysis'

In [3]:
# Import Data
customer_data = pd.read_csv(os.path.join(path, '02. Data','Original Data','customers.csv'))

In [18]:
merged_data = pd.read_pickle(os.path.join(path,'02. Data', 'Prepared Data', 'ords_prods_merge.pkl'))

# Wrangling Data & Consistency Checks

In [4]:
customer_data.head(5)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
# Drop columns

customer_data = customer_data.drop(columns = ['First Name','Surnam'])

In [6]:
# Rename columns

customer_data.rename(columns = {'STATE' : 'state'}, inplace = True)
customer_data.rename(columns = {'n_dependants' : 'dependant_count'}, inplace = True)
customer_data.rename(columns = {'fam_status' : 'marital_status'}, inplace = True)
customer_data.rename(columns = {'Gender' : 'gender'}, inplace = True)
customer_data.rename(columns = {'Age' : 'age'}, inplace = True)

In [7]:
customer_data.head(5)

,user_id,gender,state,age,date_joined,dependant_count,marital_status,income
0,26711,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Female,Maryland,26,1/1/2017,1,married,40374


In [9]:
# Check for missing data

customer_data.isnull().sum()

user_id            0
gender             0
state              0
age                0
date_joined        0
dependant_count    0
marital_status     0
income             0
dtype: int64

No missing values

In [14]:
# Check data types

customer_data.dtypes

user_id            object
gender             object
state              object
age                 int64
date_joined        object
dependant_count     int64
marital_status     object
income              int64
dtype: object

In [20]:
merged_data.dtypes

order_id                            int64
user_id                             int64
order_number                        int64
orders_day_of_week                  int64
time_of_order                       int64
days_since_prior_order            float64
product_id                          int64
add_to_cart_order                   int64
reordered                           int64
product_name                       object
aisle_id                            int64
department_id                       int64
prices                            float64
_merge                           category
busiest day                        object
busiest_days                       object
busiest_period_of_day              object
max_order                           int64
loyalty_flag                       object
mean_price                        float64
spending_flag                      object
median_days_since_prior_order     float64
frequency_flag                     object
dtype: object

In [12]:
#Changing variable data type

customer_data['user_id'] = customer_data['user_id'].astype('str')
customer_data['user_id'].dtype

dtype('O')

In [21]:
merged_data['user_id'] = merged_data['user_id'].astype('str')
merged_data['user_id'].dtype

dtype('O')

In [13]:
# Check for mix type 

for col in customer_data.columns.tolist():
  weird = (customer_data[[col]].applymap(type) != customer_data[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customer_data[weird]) > 0:
    print (col)

No mix type data

In [15]:
# Check stats

customer_data.describe()

,age,dependant_count,income
count,206209.000000,206209.000000,206209.000000
mean,49.501646,1.499823,94632.852548
std,18.480962,1.118433,42473.786988
min,18.000000,0.000000,25903.000000
25%,33.000000,0.000000,59874.000000
50%,49.000000,1.000000,93547.000000
75%,66.000000,3.000000,124244.000000
max,81.000000,3.000000,593901.000000


Everything looks correct, though I do question if max income is accurate. It is possible that someone makes that much but it seems a little high.

In [17]:
# finding duplicates

customer_data[customer_data.duplicated()]

,user_id,gender,state,age,date_joined,dependant_count,marital_status,income


No duplicates found

# Combine Data

In [22]:
#Merging datasets 
com_data = pd.merge(merged_data, customer_data, on = 'user_id', how = 'outer')

In [23]:
# Check output

com_data.head(5)

,order_id,user_id,order_number,orders_day_of_week,time_of_order,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,spending_flag,median_days_since_prior_order,frequency_flag,gender,state,age,date_joined,dependant_count,marital_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2/17/2019,3,married,40423


In [24]:
#Checking value counts
com_data['_merge'].value_counts()

both          32406041
left_only            0
right_only           0
Name: _merge, dtype: int64

full merge

# Export Data

In [25]:
com_data.to_pickle(os.path.join(path, '02. Data','Prepared Data', 'complete_data.pkl'))